In [ ]:
from google.colab import drive
import os
# Install p7zip-full for handling .7z archives
!apt-get install -y p7zip-full
!pip install nibabel nilearn dicom2nifti
from scipy.ndimage import shift

from nilearn.image import clean_img, resample_to_img
import nibabel as nib
import os
import dicom2nifti
import nibabel as nib
from nilearn.image import clean_img, resample_to_img, smooth_img, resample_img
from nilearn.masking import compute_brain_mask
from nilearn.datasets import load_mni152_template
import numpy as np
from nilearn.decomposition import CanICA
from sklearn.preprocessing import StandardScaler
from nilearn.connectome import ConnectivityMeasure
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import glob
!pip install scikit-image
from skimage.registration import phase_cross_correlation



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 84.5 MB/s eta 0:00:00


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define paths
uploaded_file_path = '/content/drive/MyDrive/schizconnect_COBRE_images_22498.7z.001'
output_dir = '/content/extracted_data'
os.makedirs(output_dir, exist_ok=True)

print("Google Drive mounted and paths defined.")


Mounted at /content/drive
Google Drive mounted and paths defined.


In [ ]:
# Rename the file to .7z
renamed_file_path = uploaded_file_path.replace('.7z.001', '.7z')
os.rename(uploaded_file_path, renamed_file_path)

print(f"File renamed to: {renamed_file_path}")


File renamed to: /content/drive/MyDrive/schizconnect_COBRE_images_22498.7z


In [ ]:
# Extract the renamed .7z file
!7z x "/content/drive/MyDrive/schizconnect_COBRE_images_22498.7z" -o"/content/extracted_data"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/                                 1 file, 2277077170 bytes (2172 MiB)

Extracting archive: /content/drive/MyDrive/schizconnect_COBRE_images_22498.7z
--
Path = /content/drive/MyDrive/schizconnect_COBRE_images_22498.7z
Type = 7z
Physical Size = 2277077170
Headers Size = 13601
Method = LZMA2:24
Solid = +
Blocks = 2

  0%      1% 309 - COBRE/sub-A00000368/ses-201101 . 00000368_ses-20110101_scans.tsv                                                                             2% 328 - COBRE/sub-A00000541/ses-20100

In [ ]:
# List the extracted files
'''for root, dirs, files in os.walk(output_dir):
    for file in files:
        print(os.path.join(root, file))'''

save_dir = '/content/fmri/corrected_nifti_data/'
os.makedirs(save_dir, exist_ok=True)

process_dir = '/content/fmri/processed_imaging_data/'
os.makedirs(process_dir, exist_ok=True)

spatial_maps_dir = '/content/fmri/spatial_maps_data/'
os.makedirs(spatial_maps_dir, exist_ok=True)
ica = CanICA(n_components=20, mask_strategy='background')
scaler = StandardScaler()

In [ ]:
import os
import glob
import nibabel as nib
import numpy as np
from nilearn.image import resample_img, resample_to_img, smooth_img, clean_img
from nilearn.masking import compute_brain_mask
from nilearn.input_data import NiftiMasker
from nilearn.datasets import load_mni152_template
from skimage.registration import phase_cross_correlation
from scipy.ndimage import shift
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
from nilearn.maskers import NiftiMasker
from nilearn.decomposition import CanICA

# Initialize scaler globally for reuse
scaler = StandardScaler()

# Motion Correction Function
def motion_correction(fmri_data, reference_volume=None):
    if reference_volume is None:
        reference_volume = np.mean(fmri_data[:5], axis=0)  # Use average of first 5 volumes as reference

    corrected_images = [reference_volume]
    reference_volume /= np.max(reference_volume)

    for i in range(1, fmri_data.shape[0]):
        fmri_data[i] /= np.max(fmri_data[i])
        fmri_data[i] = np.nan_to_num(fmri_data[i])
        shift_values, _, _ = phase_cross_correlation(reference_volume, fmri_data[i])
        corrected_image = shift(fmri_data[i], shift_values)
        corrected_images.append(corrected_image)

    corrected_data = np.stack(corrected_images, axis=0)
    return corrected_data

# Feature Extraction Function
def extract_features(nifti_img, save_dir, process_dir, i):
    #print(f"Processing file {i + 1}/{len(nifti_files)}: {nifti_img}")
    fmri_data = nifti_img.get_fdata()

    # Step 1: Motion Correction
    corrected_data = motion_correction(fmri_data)
    corrected_data[corrected_data < 0] = 0  # Set negative values to zero
    corrected_nifti_img = nib.Nifti1Image(corrected_data, nifti_img.affine)


    # Load the MNI template
    mni_template = load_mni152_template()
    downsampled_template = resample_img(
        mni_template,
        target_affine=corrected_nifti_img.affine,
        target_shape=corrected_nifti_img.shape[:3],
        interpolation="linear",
        force_resample=True,
        copy_header=True,
    )

    # Resample to template
    normalized_img = resample_to_img(corrected_nifti_img, downsampled_template, interpolation="linear", force_resample=True, copy_header=True)

    #print("Min:", np.min(normalized_img.get_fdata()), "Max:", np.max(normalized_img.get_fdata()), "Mean:", np.mean(normalized_img.get_fdata()))
    # Compute brain mask and resample
    #brain_mask = compute_brain_mask(normalized_img, lower_cutoff=0.3, upper_cutoff=0.8)
    brain_mask = mni_template.get_fdata() > 0  # Threshold the template to create a binary mask
    brain_mask_img = nib.Nifti1Image(brain_mask.astype(np.uint8), mni_template.affine)


    resampled_mask = resample_to_img(brain_mask_img, normalized_img, interpolation="nearest", force_resample=True, copy_header=True)

    # Clean the image
    cleaned_img = clean_img(
        normalized_img,
        detrend=True,
        standardize=True,
        low_pass=0.2,
        high_pass=0.005,
        t_r=1.0,
        mask_img=resampled_mask,
        n_jobs=-1,
    )

    #print("Min:", np.min(cleaned_img.get_fdata()), "Max:", np.max(cleaned_img.get_fdata()))
    # Smooth the image
    smoothed_img = smooth_img(cleaned_img, fwhm=6)
    fmri_4d_data = smoothed_img.get_fdata()  # Shape: (time_points, x_dim, y_dim, z_dim)


    # Compute the total number of voxels (x_dim * y_dim * z_dim)

    # Create a NiftiMasker
    num_voxels = np.prod(fmri_4d_data.shape[:3])  # Spatial dimensions (x, y, z)
    print(num_voxels)
    masker = NiftiMasker(standardize=True)

    # Fit the masker to your cleaned and smoothed 4D NIfTI image
    masker.fit(smoothed_img)
    #print("BYE", time_series_data.shape)

    data_2d = masker.transform(smoothed_img)  # Shape: (time_points, voxels)
    # Initialize ICA
    ica = CanICA(
        n_components=50,         # Number of components
        random_state=0,         # For reproducibility
        high_pass=0.24999970197677612,
        memory="nilearn_cache",  # Use same cache
        t_r=2.0,
        mask=masker.mask_img_  # Use the mask from NiftiMasker
    )



    # Fit ICA with the 4D NIfTI image
    ica.fit(smoothed_img)

    # Get the spatial components
    components = ica.components_

    # Spatial maps and feature extraction
    feature_vectors = []
    # Save the spatial components as individual NIfTI files

    mask = masker.mask_img_
    mask_data = mask.get_fdata()
    component_full = np.zeros(smoothed_img.shape[:3])
    for j, component in enumerate(components):
        # Create a NIfTI image for each component
        # Reshape the 1D component back into 3D space (matching the fmri data shape)
        print("component shape", component.size)

        voxel_indices = np.where(mask_data != 0)  # This gives the coordinates (i, j, k) of non-zero mask voxels

        # Place the component values into the correct 3D positions according to the mask
        component_full[voxel_indices] = component  # Assign the component values to the mask positions
        # Define a threshold dynamically based on data
        threshold = np.mean(component_full) + np.std(component_full)

        # Create binary mask
        binary_mask = (component_full > threshold).astype(int)
        binary_mask = binary_mask.astype(np.int32)  # Convert to int32


        # Generate spatial map
        spatial_map = nib.Nifti1Image(binary_mask, smoothed_img.affine)

        masker = NiftiMasker(mask_img=spatial_map, standardize=True)
        time_series = masker.fit_transform(smoothed_img)

        # Normalize time series
        time_series = scaler.fit_transform(time_series)

        # Compute voxel features
        voxel_means = np.mean(time_series, axis=0)
        voxel_stds = np.std(time_series, axis=0)

        # Combine into feature vector
        features = np.concatenate([voxel_means, voxel_stds])
        feature_vectors.append(features)

    return feature_vectors

# Main Processing Loop
def process_nifti_files(nifti_files, save_dir, process_dir):
    '''results = Parallel(n_jobs=-1)(
        delayed(extract_features)(nib.load(nii_file), save_dir, process_dir, i)
        for i, nii_file in enumerate(nifti_files)
    )'''
    results = []
    for i, nii_file in enumerate(nifti_files):
        print(f"Processing file {i + 1}/{len(nifti_files)}: {nii_file}")
        results.append(extract_features(nib.load(nii_file), save_dir, process_dir, i))
        #for i, nii_file in enumerate(nifti_files)'''
    return results

    return results
nifti_file_path = '/content/extracted_data/COBRE/sub*/ses*/func/*.nii.gz'

# Glob all files
nifti_files = glob.glob(nifti_file_path)
# Debugging step to print the files being processed
def is_empty_nifti(file_path):
    img = nib.load(file_path)
    return np.count_nonzero(img.get_fdata()) == 0  # Check if data is empty

nifti_files = glob.glob(nifti_file_path)
nifti_files = [f for f in nifti_files if not is_empty_nifti(f)]
def is_valid_nifti(file_path):
    try:
        nib.load(file_path)
        return True
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return False

nifti_files = glob.glob(nifti_file_path)
nifti_files = [f for f in nifti_files if is_valid_nifti(f)]

print("NIfTI files found:", nifti_files)

# Run the pipeline
feature_vectors = process_nifti_files(nifti_files, save_dir, process_dir)

print("Preprocessing complete!")


NIfTI files found: ['/content/extracted_data/COBRE/sub-A00023590/ses-20100101/func/sub-A00023590_ses-20100101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00037649/ses-20130101/func/sub-A00037649_ses-20130101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00027755/ses-20110101/func/sub-A00027755_ses-20110101_task-rest_run-02_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00027755/ses-20110101/func/sub-A00027755_ses-20110101_task-rest_run-01_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00000456/ses-20090101/func/sub-A00000456_ses-20090101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00028805/ses-20120101/func/sub-A00028805_ses-20120101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00031597/ses-20120101/func/sub-A00031597_ses-20120101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00037619/ses-20130101/func/sub-A00037619_ses-20130101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00024953/ses-20090

/usr/local/lib/python3.10/dist-packages/nilearn/decomposition/_base.py:492: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  self.masker_.fit()
/usr/local/lib/python3.10/dist-packages/nilearn/decomposition/canica.py:292: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  components = _MultiPCA._raw_fit(self, data)
<ipython-input-6-011e24fbd312>:114: FutureWarning: The nifti_maps_masker_ attribute is deprecated andwill be removed in Nilearn 0.11.3. Please use maps_masker_ instead.
  ica.fit(smoothed_img)


component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component shape 44169
component 

/usr/local/lib/python3.10/dist-packages/nilearn/decomposition/_base.py:492: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  self.masker_.fit()
/usr/local/lib/python3.10/dist-packages/nilearn/decomposition/canica.py:292: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  components = _MultiPCA._raw_fit(self, data)
<ipython-input-6-011e24fbd312>:114: FutureWarning: The nifti_maps_masker_ attribute is deprecated andwill be removed in Nilearn 0.11.3. Please use maps_masker_ instead.
  ica.fit(smoothed_img)


component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component shape 39114
component 

...

component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component shape 42724
component 

/usr/local/lib/python3.10/dist-packages/nilearn/decomposition/_base.py:492: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  self.masker_.fit()
/usr/local/lib/python3.10/dist-packages/nilearn/decomposition/canica.py:292: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  components = _MultiPCA._raw_fit(self, data)
<ipython-input-6-011e24fbd312>:114: FutureWarning: The nifti_maps_masker_ attribute is deprecated andwill be removed in Nilearn 0.11.3. Please use maps_masker_ instead.
  ica.fit(smoothed_img)


component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component shape 42954
component 

/usr/local/lib/python3.10/dist-packages/nilearn/decomposition/_base.py:492: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  self.masker_.fit()
/usr/local/lib/python3.10/dist-packages/nilearn/decomposition/canica.py:292: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  components = _MultiPCA._raw_fit(self, data)


KeyboardInterrupt: 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Assuming ICA features are stored in `feature_vectors`
X = np.array(feature_vectors)  # Shape: (num_samples, num_features)

# Define a simple Autoencoder class
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Initialize Autoencoder
input_dim = X.shape[1]  # Use the number of features
latent_dim = 32  # Define the desired latent dimension
autoencoder = Autoencoder(input_dim=input_dim, latent_dim=latent_dim)

# Set up optimizer and loss function
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Train Autoencoder
num_epochs = 100
for epoch in range(num_epochs):
    autoencoder.train()
    optimizer.zero_grad()
    encoded, decoded = autoencoder(torch.tensor(X, dtype=torch.float32))
    loss = criterion(decoded, torch.tensor(X, dtype=torch.float32))
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

# Extract latent features for clustering
autoencoder.eval()
with torch.no_grad():
    fmri_features, _ = autoencoder(torch.tensor(X, dtype=torch.float32))

In [ ]:
# Define the path with wildcards
beh_file_path = '/content/extracted_data/COBRE/sub*/ses*/beh/*.log'

# Use glob to match the pattern
beh_files = glob.glob(beh_file_path)
# Define the path with wildcards
tsv_file_path = '/content/extracted_data/COBRE/sub*/ses*/*.tsv'

# Use glob to match the pattern
tsv_files = glob.glob(tsv_file_path)

In [ ]:
import zipfile
# Mount Google Drive
drive.mount('/content/drive')
# Define paths
uploaded_file_path = '/content/drive/MyDrive/dataverse_files.zip'
data_dir = '/content/extracted_eeg_data'
os.makedirs(data_dir, exist_ok=True)


# Extract the contents of the zip file
with zipfile.ZipFile(uploaded_file_path, 'r') as zip_ref:
    zip_ref.extractall(data_dir)
print("Google Drive mounted and paths defined.")

Mounted at /content/drive
Google Drive mounted and paths defined.


In [ ]:
import os
import numpy as np
!pip install mne
from mne.io import read_raw_edf
from mne import Epochs, pick_types
from mne.time_frequency import psd_array_welch

# Directory containing .edf files
output_dir = "/content/eeg_features/"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Feature extraction function
def extract_features(raw, output_file, label):
    # Select EEG channels only
    raw.pick_types(eeg=True)

    # Preprocessing: Filter data (e.g., bandpass filter between 0.5-50 Hz)
    raw.filter(0.5, 50., fir_design='firwin')

    # Divide raw data into epochs (e.g., 2-second windows)
    events = np.array([[i, 0, 1] for i in range(0, int(raw.n_times), int(raw.info['sfreq'] * 2))])
    event_id = {'Epoch': 1}
    epochs = Epochs(raw, events, event_id, tmin=0, tmax=2, baseline=None, preload=True)

    # Compute power spectral density (PSD) for each epoch
    psds, freqs = psd_array_welch(
        epochs.get_data(),  # Raw data as a NumPy array
        sfreq=raw.info['sfreq'],  # Sampling frequency
        fmin=0.5,  # Minimum frequency
        fmax=50.,  # Maximum frequency
        n_fft=256  # FFT size
    )

    # Compute band-specific power features (delta, theta, alpha, beta, gamma)
    bands = {
        "delta": (0.5, 4),
        "theta": (4, 8),
        "alpha": (8, 12),
        "beta": (12, 30),
        "gamma": (30, 50)
    }
    features = []
    for band, (low, high) in bands.items():
        band_mask = (freqs >= low) & (freqs < high)  # Create mask for the band
        if np.sum(band_mask) == 0:  # Ensure the mask has valid frequencies
            raise ValueError(f"No frequencies found in the band {band}: ({low}-{high} Hz)")
        band_power = psds[:, :, band_mask].mean(axis=-1)  # Mean power for the band
        features.append(band_power)

    # Concatenate all features into a single feature vector
    features = np.hstack(features)  # Concatenate features into a 2D array
    labels = np.full((features.shape[0], 1), label)  # Shape: (num_epochs, 1)

    # Concatenate labels with features
    labeled_features = np.hstack([features, labels])  # Shape: (num_epochs, num_features + 1)

    print(f"Shape of features: {features.shape}, Label: {label}, Final shape: {labeled_features.shape}")

    # Save features to a file
    try:
        np.save(output_file, labeled_features)
        print(f"Saved features to {output_file}")
    except Exception as e:
        print(f"Failed to save features for file {file_name}: {e}")
# Loop through all .edf files in the directory
for file_name in os.listdir(data_dir):
    if file_name.endswith(".edf"):
        file_path = os.path.join(data_dir, file_name)
        print(f"Processing file: {file_path}")

        try:
            # Load the EDF file
            raw = read_raw_edf(file_path, preload=True)
            # Extract label from the filename (custom logic based on your dataset)
            if file_name.startswith('h'):
                label = 0  # Healthy
            elif file_name.startswith('s'):
                label = 1  # Sick
            else:
                print(f"Skipping file {file_name} as no label mapping is defined.")
                continue  # Skip files with unknown labels
            print(f"Assigned label {label} for file {file_name}")
            # Extract features and save to output directory
            output_file = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_features.npy")
            extract_features(raw, output_file, label=label)

        except Exception as e:
            print(f"Error processing file {file_name}: {e}")


Processing file: /content/extracted_eeg_data/s05.edf
Extracting EDF parameters from /content/extracted_eeg_data/s05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 222499  =      0.000 ...   889.996 secs...
Assigned label 1 for file s05.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
445 matching events found
No baseline cor

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Shape of features: (444, 95), Label: 1, Final shape: (444, 96)
Saved features to /content/eeg_features/s05_features.npy
Processing file: /content/extracted_eeg_data/h10.edf
Extracting EDF parameters from /content/extracted_eeg_data/h10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 278749  =      0.000 ...  1114.996 secs...
Assigned label 0 for file h10.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff freq

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Effective window size : 1.024 (s)
Shape of features: (557, 95), Label: 0, Final shape: (557, 96)
Saved features to /content/eeg_features/h10_features.npy
Processing file: /content/extracted_eeg_data/h08.edf
Extracting EDF parameters from /content/extracted_eeg_data/h08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
Assigned label 0 for file h08.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition band

...

1 bad epochs dropped
Effective window size : 1.024 (s)
Shape of features: (457, 95), Label: 0, Final shape: (457, 96)
Saved features to /content/eeg_features/h11_features.npy
Processing file: /content/extracted_eeg_data/s12.edf
Extracting EDF parameters from /content/extracted_eeg_data/s12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 271749  =      0.000 ...  1086.996 secs...
Assigned label 1 for file s12.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Shape of features: (543, 95), Label: 1, Final shape: (543, 96)
Saved features to /content/eeg_features/s12_features.npy
Processing file: /content/extracted_eeg_data/h09.edf
Extracting EDF parameters from /content/extracted_eeg_data/h09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 226249  =      0.000 ...   904.996 secs...
Assigned label 0 for file h09.edf
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff freq

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Shape of features: (452, 95), Label: 0, Final shape: (452, 96)
Saved features to /content/eeg_features/h09_features.npy


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Directory containing the feature files
output_dir = "/content/eeg_features/"

# Load features from saved .npy files
def load_features(data_dir):
    feature_list = []
    labels_list = []

    for file_name in os.listdir(data_dir):
        if file_name.endswith("_features.npy"):
            file_path = os.path.join(data_dir, file_name)
            try:
                features = np.load(file_path)
                feature_list.append(features[:, :-1])  # Exclude labels
                labels_list.append(features[:, -1])   # Labels are the last column
            except Exception as e:
                print(f"Error loading {file_name}: {e}")

    # Concatenate all the features and labels from all files
    X = np.vstack(feature_list)  # Features matrix
    y = np.hstack(labels_list)   # Labels vector

    return X, y

# Load the data
X, y = load_features(output_dir)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataLoader
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

class EEGModel(nn.Module):
    def __init__(self, eeg_input_dim, hidden_dim, output_dim):
        super(EEGModel, self).__init__()
        self.eeg_branch = nn.Sequential(
            nn.Linear(eeg_input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout layer
            nn.Linear(128, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, eeg):
        return self.eeg_branch(eeg)


# Initialize the model
model = EEGModel(eeg_input_dim=X.shape[1], hidden_dim=64, output_dim=2)  # Adjust hidden_dim as needed

# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):  # You can adjust the number of epochs
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Evaluate the model
model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.numpy())
        y_true.extend(labels.numpy())

# Print classification report
print(classification_report(y_true, y_pred))

# Optional: Save the trained model if needed
import joblib
joblib.dump(clf, 'eeg_classifier_model.pkl')


In [ ]:
class FusionModel(nn.Module):
    def __init__(self, eeg_dim, fmri_dim, fusion_dim, output_dim):
        super(FusionModel, self).__init__()
        self.eeg_branch = EEGModel(eeg_dim, hidden_dim=128, output_dim=64)  # EEG encoder
        self.fmri_branch = nn.Linear(fmri_dim, 64)  # fMRI latent representation
        self.fusion_layer = nn.Sequential(
            nn.Linear(64 + 64, fusion_dim),
            nn.ReLU(),
            nn.Linear(fusion_dim, output_dim)
        )

    def forward(self, eeg, fmri):
        eeg_features = self.eeg_branch(eeg)
        fmri_features = self.fmri_branch(fmri)
        combined_features = torch.cat((eeg_features, fmri_features), dim=1)
        output = self.fusion_layer(combined_features)
        return output
fusion_model = FusionModel(eeg_dim=X.shape[1], fmri_dim=128, fusion_dim=128, output_dim=2)
optimizer = optim.Adam(fusion_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(50):
    optimizer.zero_grad()
    outputs = fusion_model(X_train, fmri_features[:len(X_train)])  # EEG (labeled) + fMRI (unlabeled)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()


# Evaluate the model
fusion_model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for i in range(0, len(X_test), 64):  # Assuming batch size of 64
        # Make sure we correctly handle the test data and fMRI features
        eeg_batch = X_test[i:i+64]
        fmri_batch = fmri_features[len(X_train)+i:len(X_train)+i+64]  # fMRI for test set
        labels_batch = y_test[i:i+64]

        # Forward pass
        outputs = fusion_model(eeg_batch, fmri_batch)
        _, predicted = torch.max(outputs, 1)  # Get the class with highest probability

        y_pred.extend(predicted.numpy())
        y_true.extend(labels_batch.numpy())

# Print classification report
print(classification_report(y_true, y_pred))

In [ ]:
# Step 1: Load data from file
# Replace 'your_file.csv' with the actual path to your file (e.g., 'data/eeg_data.csv')
data = pd.read_csv('your_file.csv')

# Step 2: Drop categorical columns
eeg_features = data.drop(columns=['group', 'UHR-status'])

# Step 3: Normalize the features using StandardScaler
scaler = StandardScaler()
eeg_features_scaled = scaler.fit_transform(eeg_features)

# Step 4: View the scaled EEG features
print("Scaled EEG Features:")
print(pd.DataFrame(eeg_features_scaled, columns=eeg_features.columns))


In [ ]:

# Simulated Data (EEG and fMRI features)
eeg_data = pd.DataFrame({
    'GLX': [5.969, 5.490, 5.556, 5.887, 4.845, 5.783],
    'GABA': [2.155, 2.675, 2.349, 4.019, 2.150, 2.140],
    'RATIO': [2.770, 2.053, 2.366, 1.465, 2.254, 2.702],
    'RMOG': [6.60E-05, 4.99E-05, 4.76E-05, 3.58E-05, 4.43E-05, 4.83E-05],
    'LMOG': [8.22E-04, 6.74E-04, 4.55E-04, 6.67E-04, 5.96E-04, 5.49E-04],
    'RCAL': [6.90E-05, 5.06E-05, 6.21E-05, 5.18E-05, 6.04E-05, 4.30E-05],
    'LCAL': [2.73E-03, 1.25E-03, 1.34E-03, 5.40E-04, 1.79E-03, 1.12E-03]
})

# Normalize EEG Data
eeg_features = tf.keras.utils.normalize(eeg_data.values, axis=0)

# Simulated fMRI Data (1D Features)
fmri_data = np.random.rand(len(eeg_features), 64)  # Assume 64-dimensional fMRI feature space

# Simulated Labels (Disease Diagnosis: 0 = healthy, 1 = disease)
labels = np.array([0, 1, 1, 0, 1, 0])

# Define Generator Model (e.g., EEG -> fMRI)
def build_generator(input_shape, output_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Dense(128, activation='relu')(inputs)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(output_shape, activation='linear')(x)  # Map to target dimension
    return models.Model(inputs, outputs, name="Generator")

# Define Discriminator Model (e.g., Discriminate fMRI features)
def build_discriminator(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Dense(128, activation='relu')(inputs)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)  # Binary classification: Real or Fake
    return models.Model(inputs, outputs, name="Discriminator")

# Define Intermediate Fusion Model
def build_intermediate_fusion_model(eeg_features, fmri_features):
    fusion = layers.Concatenate()([eeg_features, fmri_features])
    fusion = layers.Dense(128, activation='relu')(fusion)
    fusion = layers.Dense(64, activation='relu')(fusion)
    return fusion

# Hyperparameters
input_dim = eeg_features.shape[1]  # EEG feature dimension
target_dim = fmri_data.shape[1]    # fMRI feature dimension
batch_size = 4
epochs = 50

# Build Models
G_EEG_to_fMRI = build_generator((input_dim,), target_dim)  # Generator: EEG -> fMRI
G_fMRI_to_EEG = build_generator((target_dim,), input_dim)  # Generator: fMRI -> EEG
D_fMRI = build_discriminator((target_dim,))  # Discriminator for fMRI features
D_EEG = build_discriminator((input_dim,))  # Discriminator for EEG features

# Fusion Layers: Combine both domain features before final output layers
intermediate_fusion_eeg_fmri = build_intermediate_fusion_model(
    G_EEG_to_fMRI.output, G_fMRI_to_EEG.output
)

# Define a Classifier Model (Brain Disease Diagnosis)
def build_classifier(fusion_layer):
    x = layers.Dense(64, activation='relu')(fusion_layer)
    x = layers.Dense(32, activation='relu')(x)
    output = layers.Dense(1, activation='sigmoid')(x)  # Binary classification: 0 = Healthy, 1 = Disease
    return models.Model(inputs=[G_EEG_to_fMRI.input, G_fMRI_to_EEG.input], outputs=output)

# Build the full model including the classifier
classifier = build_classifier(intermediate_fusion_eeg_fmri)

# Compile the model
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
# Use EEG features and fMRI features as inputs
classifier.fit([eeg_features, fmri_data], labels, epochs=epochs, batch_size=batch_size, validation_split=0.2)

# Evaluate the model on test data (simulated here)
test_eeg_features = np.random.rand(10, input_dim)  # Simulated test EEG features
test_fmri_features = np.random.rand(10, target_dim)  # Simulated test fMRI features
test_labels = np.array([1, 0, 1, 0, 0, 1, 1, 0, 0, 1])  # Simulated test labels

# Evaluate on test set
test_loss, test_acc = classifier.evaluate([test_eeg_features, test_fmri_features], test_labels)
print(f"Test accuracy: {test_acc * 100:.2f}%")



In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
import os

# Directory containing the .npy files
#data_dir = "path_to_your_directory"  # Replace with the actual path to your directory

# Step 1: List all .npy files in the directory
data_files = [os.path.join(output_dir, file) for file in os.listdir(output_dir) if file.endswith(".npy")]

# Step 2: Load all datasets and combine them
data = np.vstack([np.load(file) for file in data_files])

# Step 3: Separate Features and Labels
features = data[:, :-1]  # All columns except the last one
labels = data[:, -1]     # The last column contains labels

# Print shapes to confirm
print(f"Features shape: {features.shape}")
print(f"Labels shape: {labels.shape}")


# Normalize the features
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

# Convert the features and labels to PyTorch tensors
features_tensor = torch.tensor(features_normalized, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features_tensor, labels_tensor, test_size=0.2, random_state=42)

# Define the FusionModel (EEG + fMRI)
class FusionModel(nn.Module):
    def __init__(self, eeg_input_dim, fmri_input_dim, hidden_dim, output_dim):
        super(FusionModel, self).__init__()
        # EEG branch
        self.eeg_branch = nn.Sequential(
            nn.Linear(eeg_input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        # fMRI branch
        self.fmri_branch = nn.Sequential(
            nn.Linear(fmri_input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU()
        )
        # Fusion layers
        self.fusion_layer = nn.Sequential(
            nn.Linear(64 + 256, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, eeg, fmri):
        eeg_features = self.eeg_branch(eeg)
        fmri_features = self.fmri_branch(fmri)
        combined_features = torch.cat((eeg_features, fmri_features), dim=1)
        output = self.fusion_layer(combined_features)
        return output

# Initialize the model
eeg_input_dim = features.shape[1]  # Number of EEG features
fmri_input_dim = 1000  # Number of fMRI features (you can modify this based on your dataset)
hidden_dim = 128  # Hidden dimension in the fusion layer
output_dim = 2  # Number of classes (binary classification)

model = FusionModel(eeg_input_dim, fmri_input_dim, hidden_dim, output_dim)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train, X_train)  # Here, X_train is passed as both eeg and fmri (adjust as necessary)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_test, X_test)  # Here, X_test is passed as both eeg and fmri (adjust as necessary)
    predictions = torch.argmax(outputs, dim=1)
    accuracy = (predictions == y_test).float().mean().item()
    print(f"Test Accuracy: {accuracy:.2f}")
